# Soil texture classes


This Notebook illustrates how the percentages of the particle-size classes sand, silt, and clay determine the soil texture and how these can be read from a soil texture triangle (see e.g. FAO 2006).

### Dependensies/Installation

This Notebook uses mplotern as a matplotlib based python library to visualise soil texture classes as a ternary plot.
To install mplotern see the following link:
https://mpltern.readthedocs.io/en/latest/installation.html#

In [3]:
import numpy as np
from ipywidgets import *
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator, MultipleLocator
from matplotlib._cm import _Set3_data
from mpltern.datasets import soil_texture_classes
from IPython.display import display, clear_output

# Create a button to restart the task
restart_button = widgets.Button(description="Restart", layout=widgets.Layout(display='none'))

def calculate_centroid(vertices):
    """Calculate the centroid of a polygon."""
    roll0 = np.roll(vertices, 0, axis=0)
    roll1 = np.roll(vertices, 1, axis=0)
    cross = np.cross(roll0, roll1)
    area = 0.5 * np.sum(cross)
    return np.sum((roll0 + roll1) * cross[:, None], axis=0) / (6.0 * area)

def plot_soil_texture_classes(ax):
    """Plot soil texture classes."""
    classes = soil_texture_classes
    for (key, value), color in zip(classes.items(), _Set3_data):
        tn0, tn1, tn2 = np.array(value).T
        patch = ax.fill(tn0, tn1, tn2, ec="k", fc=color, alpha=0.6, zorder=2.1)
        centroid = calculate_centroid(patch[0].get_xy())
        label = key[::-1].replace(" ", "\n", 1)[::-1].capitalize()
        ax.text(centroid[0], centroid[1], label, ha="center", va="center", transform=ax.transData)

    ax.taxis.set_major_locator(MultipleLocator(10.0))
    ax.laxis.set_major_locator(MultipleLocator(10.0))
    ax.raxis.set_major_locator(MultipleLocator(10.0))
    ax.taxis.set_minor_locator(AutoMinorLocator(2))
    ax.laxis.set_minor_locator(AutoMinorLocator(2))
    ax.raxis.set_minor_locator(AutoMinorLocator(2))
    ax.grid(which="major", linewidth=1)
    ax.grid(which='minor', linewidth=0.4)
    ax.set_tlabel("Clay (%)")
    ax.set_llabel("Sand (%)")
    ax.set_rlabel("Silt (%)")
    ax.taxis.set_ticks_position("tick2")
    ax.laxis.set_ticks_position("tick2")
    ax.raxis.set_ticks_position("tick2")

def plot_rand_point(ax):
    """Plot a random point in the ternary diagram and return its values."""
    clay = np.random.randint(0, 21) * 5
    sand = np.random.randint(0, (100 - clay) // 5 + 1) * 5
    silt = 100 - clay - sand
    ax.plot(clay, sand, silt, 'ro', label='Random Point', zorder=2.5)
    return [clay, sand, silt]

def catch_the_point(inp_clay, inp_sand, inp_silt, ax, fig, rand_values):
    """Check the user input and see if it matches the random point."""
    inp_values = [inp_clay, inp_sand, inp_silt]
    correct_guess = all([inp_values[i] == rand_values[i] for i in range(3)])

    with text_output:
        clear_output(wait=True)  # Clear previous messages

        if correct_guess:
            print('Congratulations!')
            fig_name=f"{inp_clay}%clay_{inp_silt}%silt_{inp_sand}%sand.png"
            fig.savefig(fig_name)

            restart_button.layout.display = 'inline-block'
        else:
            print(f'Your guess for %Clay: {inp_clay} is {"correct" if inp_clay == rand_values[0] else "incorrect"}\n\n'
                  f'Your guess for %Sand: {inp_sand} is {"correct" if inp_sand == rand_values[1] else "incorrect"}\n\n'
                  f'Your guess for %Silt: {inp_silt} is {"correct" if inp_silt == rand_values[2] else "incorrect"}\n\nPlease try again.')

        # Ensure the plot is fully updated
        plt.show()  # This explicitly renders the plot
        plt.pause(0.001)  # Give the plot time to update

def on_restart_button_click(b):

    """Clear the plot and restart the task."""
    with output:
        clear_output(wait=True)  # Delete the output before displaying a new one.
        plt.close('all') 
        fig, ax = plt.subplots(figsize=(8,8), subplot_kw={'projection': 'ternary', 'ternary_sum': 100.0})   # New Figure
        plot_soil_texture_classes(ax)
        
        global rand_values  # New random values
        rand_values = plot_rand_point(ax)  # A new random point
        
        restart_button.layout.display = 'none'  # Hide the restart button again
        plt.show()

    # Reset the input values to 0
    inp_clay.value = 0
    inp_sand.value = 0
    inp_silt.value = 0

    with text_output:
        clear_output(wait=True)  # Clear the text output
        print("Game restarted! Please enter new values.")
    
    # Connect the new values to the catch_the_point funktion
    out_sol = widgets.interactive_output(
        catch_the_point,
        {'inp_clay': inp_clay, 'inp_sand': inp_sand, 'inp_silt': inp_silt, 
         'ax': widgets.fixed(ax), 'fig':widgets.fixed(fig), 'rand_values': widgets.fixed(rand_values)}
    )

def main():
    """Main function to run the game."""
    # Widgets for interactive input
    global inp_clay, inp_sand, inp_silt
    inp_clay = widgets.BoundedIntText(value=0, min=0, max=100, step=1, description='% Clay:')
    inp_sand = widgets.BoundedIntText(value=0, min=0, max=100, step=1, description='% Sand:')
    inp_silt = widgets.BoundedIntText(value=0, min=0, max=100, step=1, description='% Silt:')

    # Output-Widget for the
    global output, text_output
    output = widgets.Output()
    text_output = widgets.Output()

    with output:
        # Initial plot
        fig, ax = plt.subplots(figsize=(8,8),subplot_kw={'projection': 'ternary', 'ternary_sum': 100.0})
        plot_soil_texture_classes(ax)

        # Plot a random point and return the values
        global rand_values
        rand_values = plot_rand_point(ax)

        plt.show()

    # Bind the restart button to the click event
    restart_button.on_click(on_restart_button_click)

    # Interactive output for the guesses
    out_sol = widgets.interactive_output(
        catch_the_point,
        {'inp_clay': inp_clay, 'inp_sand': inp_sand, 'inp_silt': inp_silt, 
         'ax': widgets.fixed(ax), 'fig':widgets.fixed(fig), 'rand_values': widgets.fixed(rand_values)}
        )

    # Text and Layout
    Title = HTML('<h2>Soil Textures<h2>')

    Text = VBox([HTML('On the right, you can see the plot of a ternery soil texture diagram. '
                          'Percentages of clay, silt, and sand for a given soil are indicated on each axis. '
                          'Each soil texture is represented by a polygon corresponding to its composition. '
                          'The red dot in the diagram represents the composition of a given soil sample. '
                          'Your task is to fill in the correct percentages of clay, silt, and sand for this soil sample. '
                          'Once you have completed the task correctly, you can restart and try again. ')])
    
    userInput = widgets.VBox([Label(' '), inp_clay, inp_sand, inp_silt, Label(' ')])
    
    grid = GridspecLayout(18,3, height='750px')
    grid[:1, :] = Title
    grid[1:3, :] = Text
    grid[4:8, :1] = userInput
    grid[8:14, :1] = text_output
    grid[14:16, :1] = restart_button
    grid[16:18, :1] = out_sol
    grid[3:,1:] = output

    display(grid)

if __name__ == "__main__":
    main()


GridspecLayout(children=(HTML(value='<h2>Soil Textures<h2>', layout=Layout(grid_area='widget001')), VBox(child…

### References:

FAO (2006): Guidelines for soil description. Fourth edition. Food and Agricultural Organization of the United Nations, Rome. Available at: https://openknowledge.fao.org/handle/20.500.14283/a0541e

Ikeda, Yuji (2024): yuzie007/mpltern: 1.0.4 (1.0.4). Zenodo. https://doi.org/10.5281/zenodo.11068993

This work &copy; 2024 by Edith Grießer, Steffen Birk (University of Graz) is licensed under  <a href="https://creativecommons.org/licenses/by/4.0/?ref=chooser-v1" target="_blank" rel="license noopener noreferrer" style="display:inline-block;">CC BY 4.0<img style="height:22px!important;margin-left:3px;vertical-align:text-bottom;" src="https://mirrors.creativecommons.org/presskit/icons/cc.svg?ref=chooser-v1" alt=""><img style="height:22px!important;margin-left:3px;vertical-align:text-bottom;" src="https://mirrors.creativecommons.org/presskit/icons/by.svg?ref=chooser-v1" alt=""></a></p> 